<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import pandas as pd
import numpy as np
import pyodbc
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook
from matplotlib.ticker import ScalarFormatter
import re
import seaborn as sns
pd.set_option('display.max_columns', 500)
sns.set()

In [3]:
# Connect to Access Database and access data with SQL commands
access_database = r'C:\Users\thwhi\Dropbox\Personal\Programming\Aircraft Stuff\avall_full_aircraft_crash_database\avall.mdb'
driver = r'Microsoft Access Driver (*.mdb, *.accdb)'
conn = pyodbc.connect(r'DRIVER={' + driver + '};DBQ=' + access_database + ';')

In [4]:
df_events = pd.read_sql_query('SELECT * FROM events',conn)
df_aircraft = pd.read_sql_query('SELECT * FROM aircraft',conn)

In [5]:
df_tail_master = pd.read_csv(r'C:\Users\thwhi\Dropbox\Personal\Programming\Aircraft Stuff\Aircraft Tail Number Database\master.txt')

C:\Users\thwhi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Function to remove duplicates, set all values to uppercase, and to strip all entries of leading and trailing spaces
def strip_strings(df):
    df = df.loc[:,~df.columns.duplicated()]
    df = df.dropna(axis=1,how='all')
    for col in df.columns:
        if df[col].dtype=='O':
            df[col] = df[col].str.strip()
        if df[col].dtype=='O':
            df[col] = df[col].str.upper()
    return(df)

In [7]:
df_events = strip_strings(df_events)
df_aircraft = strip_strings(df_aircraft)

In [8]:
df_tail_master = strip_strings(df_tail_master)
df_tail_master = df_tail_master.drop_duplicates('SERIAL NUMBER')

In [9]:
sum(df_aircraft['acft_serial_no'].duplicated())

19383

In [10]:
len(df_aircraft)

83169

In [11]:
df_events

,ev_id,ntsb_no,ev_type,ev_date,ev_dow,ev_time,ev_tmzn,ev_city,ev_state,ev_country,ev_site_zipcode,ev_year,ev_month,mid_air,on_ground_collision,latitude,longitude,latlong_acq,apt_name,ev_nr_apt_id,ev_nr_apt_loc,apt_dist,apt_dir,apt_elev,wx_brief_comp,wx_src_iic,wx_obs_time,wx_obs_dir,wx_obs_fac_id,wx_obs_elev,wx_obs_dist,wx_obs_tmzn,light_cond,sky_cond_nonceil,sky_nonceil_ht,sky_ceil_ht,sky_cond_ceil,vis_rvr,vis_rvv,vis_sm,wx_temp,wx_dew_pt,wind_dir_deg,wind_dir_ind,wind_vel_kts,wind_vel_ind,gust_ind,gust_kts,altimeter,wx_dens_alt,wx_int_precip,metar,ev_highest_injury,inj_f_grnd,inj_m_grnd,inj_s_grnd,inj_tot_f,inj_tot_m,inj_tot_n,inj_tot_s,inj_tot_t,invest_agy,ntsb_docket,ntsb_notf_from,ntsb_notf_date,ntsb_notf_tm,fiche_number,lchg_date,lchg_userid,wx_cond_basic,faa_dist_office
0,20001208X07734,LAX97FA143,ACC,1997-04-02,WE,2016.0,MST,GRAND CANYON,AZ,USA,86023,1997,4,N,N,,,None,,,OFAP,NaN,NaN,NaN,PARP,WFAC,1954.0,180.0,GCN,6606.0,21.0,MST,NDRK,SCAT,3600.0,6000.0,BKN,0.0,0.0,10.0,-2.0,-3.0,0.0,U,0.0,CALM,N,0.0,29.000000,NaN,UNK,None,FATL,0.0,0.0,0.0,2.0,NaN,NaN,NaN,2.0,N,1890.0,None,1997-04-02,1922.0,IMAGE,2000-12-08 12:13:18,DBO,IMC,None
1,20021008X05297,CHI02LA299,ACC,2002-09-29,SU,2100.0,CDT,MOORHEAD,MN,USA,56560,2002,9,N,N,465021N,0963947W,None,MOORHEAD MUNICIPAL,JKJ,ONAP,NaN,NaN,917.0,UNK,WFAC,2053.0,47.0,FAR,908.0,9.0,BST,NDRK,CLER,NaN,1300.0,OVC,NaN,NaN,7.0,14.0,12.0,160.0,Y,13.0,SPEC,Y,0.0,29.709999,NaN,None,None,SERS,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0,N,13246.0,FAA,2002-10-03,945.0,DMS,2003-04-23 12:26:43,NTSB\MONR,VMC,MINNEAPOLIS FSDO
2,20040127X00111,LAX04LA103,ACC,2003-10-12,SU,1640.0,MST,SCOTTSDALE,AZ,USA,85255,2003,10,N,N,333722N,1115438W,None,SCOTTSDALE,SDL,ONAP,NaN,NaN,1510.0,FULL,WFAC,1653.0,NaN,KSDL,1510.0,NaN,MST,DAYL,CLER,NaN,NaN,NONE,NaN,NaN,10.0,33.0,12.0,NaN,None,NaN,CALM,N,NaN,29.889999,NaN,None,None,NONE,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,N,17586.0,FAA,2004-01-22,1215.0,DMS,2005-04-28 14:59:26,NTSB\JOHB,VMC,WP-SDL-FSDO
3,20181125X85246,GAA19CA073,ACC,2018-11-25,SU,1320.0,PST,PORTERVILLE,CA,USA,93257,2018,11,N,N,360148N,1190347W,EST,PORTERVILLE MUNI,PTV,ONAP,0.0,327.0,443.0,None,WFAC,2156.0,177.0,KPTV,442.0,0.0,UTC,DAYL,CLER,NaN,NaN,NONE,NaN,NaN,10.0,21.0,12.0,300.0,Y,6.0,SPEC,N,NaN,30.059999,NaN,None,METAR KPTV 252156Z AUTO 30006KT 10SM CLR 21/12...,NONE,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,N,NaN,NTSB_WPR,NaT,NaN,None,2018-12-19 16:51:35,GUTE,VMC,None
4,20001213X34369,ANC86LA135,ACC,1986-08-23,SA,1100.0,AKD,TALKEETNA,AK,USA,99676,1986,8,N,N,,,None,,,OFAP,0.0,0.0,0.0,UNK,PILO,0.0,0.0,,0.0,0.0,,DAYL,CLER,0.0,0.0,NONE,0.0,0.0,50.0,10.0,-18.0,0.0,U,0.0,UNK,U,0.0,NaN,NaN,UNK,None,NONE,0.0,0.0,0.0,NaN,NaN,3.0,NaN,NaN,N,891.0,None,1986-08-23,1430.0,31238,2001-01-02 09:14:50,DBO,VMC,None
5,20001213X35290,ANC87LA019,ACC,1986-12-07,SU,1256.0,AST,SHAGELUK,AK,USA,99665,1986,12,N,N,,,None,,1,OFAP,0.0,0.0,0.0,UNK,PILO,0.0,0.0,,0.0,0.0,,DAYL,SCAT,1500.0,0.0,NONE,0.0,0.0,40.0,-1.0,-1.0,0.0,U,0.0,CALM,N,0.0,29.000000,NaN,LGT,None,NONE,0.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,N,1400.0,None,1986-12-07,1300.0,31791,2001-01-02 09:14:55,DBO,VMC,None
6,20001214X43255,CHI83LA252,ACC,1983-06-12,SU,1300.0,CDT,BLACKDUCK,MN,USA,56630,1983,6,N,N,,,None,,,OFAP,NaN,NaN,NaN,UNK,WFAC,1245.0,225.0,BJI,1390.0,20.0,CDT,DAYL,CLER,0.0,2000.0,BKN,0.0,0.0,7.0,22.0,17.0,180.0,U,13.0,UNK,N,0.0,29.000000,NaN,UNK,None,NONE,0.0,0.0,0.0,NaN,NaN,2.0,NaN,NaN,N,1284.0,None,NaT,0.0,21578,2001-01-02 09:15:51,DBO,VMC,None
7,20010110X00143,CHI86LA076,ACC,1986-01-23,TH,1615.0,EST,INDIANAPOLIS,IN,USA,46236,1986,1,N,N,,,None,MT. COMFORT,2IN2,ONAP,0.0,0.0,860.0,UNK,PILO,0.0,0.0,,0.0,0.0,,DAYL,CLER,0.0,0.0,NONE,0.0,0.0,15.0,6.0,NaN,0.0,U,0.0,CALM,N,0.0,NaN,NaN,UNK,None,NONE,0.0,0.0,0.0,NaN,NaN,1.0,NaN,NaN,N,90.0,None,1986-01-27,1520.0,29891,2001-01-10 14:57:03,DBO,VMC,None
8,20001213X33703,SEA86LA119,ACC,1986-05-06,TU,1815.0,PDT,YAKIMA,WA,USA,98903,1986,5,N,N,,,None,YAKIMA AIR TERMINAL,YKM,ONAP,0.0,0.0,1095.0,UNK,WFAC,1750.0,90.0,YKM,1095.0,1.0,PDT,DAYL,SCAT,5000.0,25000.0,BKN,0.0,

In [12]:
df_events = df_events[df_events['ev_country']=='USA']


In [13]:
events_cols_to_keep = ['ev_id','ntsbo_no','ev_type','ev_date','ev_time','']

In [14]:
df_aircraft.head()

,ev_id,Aircraft_Key,regis_no,ntsb_no,acft_missing,far_part,flt_plan_filed,flight_plan_activated,damage,acft_fire,acft_expl,acft_make,acft_model,acft_series,acft_serial_no,cert_max_gr_wt,acft_category,acft_reg_cls,homebuilt,fc_seats,cc_seats,pax_seats,total_seats,num_eng,fixed_retractable,type_last_insp,date_last_insp,afm_hrs_last_insp,afm_hrs,elt_install,elt_oper,elt_aided_loc_ev,elt_type,owner_acft,owner_street,owner_city,owner_state,owner_country,owner_zip,oper_individual_name,oper_name,oper_same,oper_dba,oper_addr_same,oper_street,oper_city,oper_state,oper_country,oper_zip,oper_code,certs_held,oprtng_cert,oper_cert,oper_cert_num,oper_sched,oper_dom_int,oper_pax_cargo,type_fly,second_pilot,dprt_pt_same_ev,dprt_apt_id,dprt_city,dprt_state,dprt_country,dprt_time,dprt_timezn,dest_same_local,dest_apt_id,dest_city,dest_state,dest_country,phase_flt_spec,report_to_icao,evacuation,lchg_date,lchg_userid,afm_hrs_since,rwy_num,rwy_len,rwy_width,site_seeing,air_medical,med_type_flight,acft_year,commercial_space_flight,unmanned,ifr_equipped_cert,elt_mounted_aircraft,elt_connected_antenna
0,20001208X07734,1,N6172C,LAX97FA143,N,091,IFR,None,DEST,NONE,NONE,CESSNA,T210N,T210N,210-63820,3800.0,AIR,USUS,N,NaN,NaN,NaN,6.0,1.0,RETR,ANNL,1996-11-16,34.0,1643.0,Y,U,U,None,JAMES E. ELDREDGE,4039 KNOLL RIDGE AVE.,LAS VEGAS,NV,,89030,None,,Y,RANGER ENTERPRISES,Y,,,,,,,N,N,UNK,None,None,None,None,PERS,N,N,LVS,LAS VEGAS,NM,,1750.0,MST,LOCL,VGT,,NV,,540.0,None,None,2001-01-02 10:34:39,DBO,None,0,NaN,NaN,N,N,None,NaN,False,False,False,False,False
1,20021008X05297,1,N9714W,CHI02LA299,None,091,NONE,N,SUBS,NONE,NONE,PIPER,PA-28-140,None,28-23190,2150.0,AIR,USUS,N,NaN,NaN,NaN,4.0,1.0,None,None,NaT,NaN,4700.0,Y,U,N,None,WEST CENTRAL AIRWAYS INC,4005 HIGHWAY 210 W,FERGUS FALLS,MN,USA,56537,N,None,Y,None,Y,None,None,None,None,None,IIQA,N,None,None,None,None,None,None,INST,Y,N,04Y,HAWLEY,MN,USA,2045.0,CDT,SAME,JKJ,MOOREHEAD,MN,USA,570.0,None,N,2003-03-13 11:39:24,SULP,ACCI,12,4000.0,75.0,N,N,None,NaN,False,False,False,False,False
2,20040127X00111,1,N7127V,LAX04LA103,None,091,VFR,Y,SUBS,NONE,NONE,MOONEY,20F,None,22-0056,2740.0,AIR,USUS,N,NaN,NaN,NaN,4.0,1.0,RETR,ANNL,2003-01-10,120.0,4120.0,Y,N,N,None,THOMAS CHENAL,None,SCOTTSDALE,AZ,USA,85254,N,None,Y,None,Y,None,None,None,None,None,None,N,None,None,None,None,None,None,PERS,N,N,SFA,SANTA FE,NM,USA,1500.0,CST,SAME,SDL,SCOTTSDALE,AZ,USA,570.0,N,N,NaT,None,ACCI,21,8251.0,75.0,N,N,None,NaN,False,False,False,False,False
3,20181125X85246,1,N66710,GAA19CA073,None,091,NONE,U,SUBS,NONE,NONE,BEECH,C23,None,M-2387,2450.0,AIR,None,N,2.0,NaN,2.0,4.0,1.0,None,ANNL,2017-11-20,NaN,6109.0,Y,N,U,C91,"DONALD UTTENREITHER, RICKEY COOKSEY",None,PORTERVILLE,CA,USA,932579130,Y,RICKEY COOKSEY,N,None,Y,None,PORTERVILLE,CA,USA,932579130,None,Y,None,None,None,None,None,None,PERS,N,N,PTV,PORTERVILLE,CA,USA,1250.0,PST,LOCL,PTV,PORTERVILLE,CA,USA,NaN,None,None,2018-12-21 17:23:10,GUTE,INSP,30,5960.0,150.0,N,N,None,1983.0,False,False,False,False,False
4,20001213X34369,1,N70021,ANC86LA135,N,091,NONE,None,SUBS,NONE,NONE,CESSNA,A-185E,A-185E,18501896,3300.0,AIR,USUS,N,NaN,NaN,NaN,6.0,1.0,None,UNK,NaT,NaN,962.0,Y,N,N,None,LARRY D. GRANT,P.O. BOX 772694,EAGLE RIVER,AK,,99577,None,,Y,,Y,,,,,,,N,N,UNK,None,None,None,None,PERS,N,N,MRI,ANCHORAGE,AK,,1000.0,ADT,None,,CHICKLOON RIVER,AK,,570.0,None,None,2001-01-02 10:36:05,DBO,None,0,0.0,0.0,N,N,None,NaN,False,False,False,False,False


In [16]:
df = df_aircraft[['ev_id','acft_make','acft_model']]
df['make_model'] = df_aircraft['acft_make'] + ' ' + df_aircraft['acft_model']

C:\Users\thwhi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ev_id,acft_make,acft_model,make_model
0,20001208X07734,CESSNA,T210N,CESSNA T210N
1,20021008X05297,PIPER,PA-28-140,PIPER PA-28-140
2,20040127X00111,MOONEY,20F,MOONEY 20F
3,20181125X85246,BEECH,C23,BEECH C23
4,20001213X34369,CESSNA,A-185E,CESSNA A-185E
